In [10]:
import numpy as np
import time

In [11]:
def my_vectorize(func):
    def vecfunc(*args, **kwargs):
        
        return [func(a, *args[1:], **kwargs) for a in args[0]]
    
    return vecfunc

In [12]:
import time

In [28]:
def timeit(func):
    def modfunc(*args, **kwargs):
        start = time.time()
        ret_val = func(*args, **kwargs)
        end = time.time()
        print(f'Time elapsed: {end-start:.2f}s')
        
        return ret_val
    
    return modfunc

In [14]:
def na_patch(exception_return='first'):
    def middle(func):
        def modfunc(*args, **kwargs):
            try:
                return func(*args, **kwargs)
        
            except Exception:
                if exception_return == 'first':
                    return args[0]
                
                elif callable(exception_return):
                    return exception_return(*args, **kwargs)
                
                else:
                    return exception_return
                    
        return modfunc
        
    return middle

In [15]:

@na_patch(exception_return=lambda x: x**2 if type(x) is int else np.nan)
def strip_wspace(s):
    # Do something more here
    return s.strip()


In [30]:
@timeit
def calculate_function(vals, method, base_func):
    if method == 'list_comp':
        return np.array([base_func(v) for v in vals])
    
    if method == 'map':
        return np.array(list(map(base_func, vals)))
    
    if method == 'np_vec':
        func_vectorized = np.vectorize(base_func)
        return func_vectorized(vals)
    
    if method == 'generate_map':
        uniq = np.unique(vals)
        uniq_comp = calculate_function(uniq, 'list_comp', base_func)
        map_dict = dict(zip(uniq, uniq_comp))
        lookup =  np.vectorize(lambda x: map_dict[x])
    
        return lookup(vals)
        
    

In [31]:
vals = np.random.randint(1, 100, 1000000)

In [32]:
def func(x):
    if x >5 :
        return x**2
    else:
        return 3*x + np.sin(x)

In [33]:
# %%time
v1 = calculate_function(vals, 'np_vec', func);

Time elapsed: 0.49s


In [34]:
v2 = calculate_function(vals, 'list_comp', func )

Time elapsed: 0.97s


In [35]:
v3 = calculate_function(vals, 'generate_map', func)

Time elapsed: 0.00s
Time elapsed: 0.35s


In [37]:
v4 = calculate_function(vals, 'map', func)

Time elapsed: 1.62s


In [40]:
%%time
sign_lookup = np.vectorize(lambda x: 1 if x > 0 else 0)
v5 = sign_lookup(vals-5)*(vals**2) + 3*vals + np.sin(vals)

Wall time: 171 ms
